# Copy on Leon's

### Random Forest Classification

In [105]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

In [84]:
def model_training(clf,X_train,y_train):
    """
    Input  : LSI vectors
    Purpose: train classification model
    Output : return trained classifier
    """
    clf.fit(X_train,y_train)
    return clf

def model_predictions(clf, X_test):
    """
    Input  : trained classifier model, the test set
    Purpose: make predictions on test data
    Output : predictions
    """
    y_pred=clf.predict(X_test)
    
    return y_pred

def reco_rate(ref_labels, predicted_labels):
    commun_labels = (pd.np.array(ref_labels)==pd.np.array(predicted_labels)).sum()
    return 100 * commun_labels / len(ref_labels)

Hyperparameters to tune:

In [106]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
print(random_grid)

{'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]}


### LSI

Use the random grid to search for best hyperparameters:

In [107]:
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(np.array(lsi_model[corpus_TFmatrix_train])[:,:,1], y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed:   41.2s
[Parallel(n_jobs=-1)]: Done 146 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed:  6.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
               

In [112]:
rf_random.best_params_

{'n_estimators': 200,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'auto',
 'max_depth': 90,
 'bootstrap': False}

Copying the parameters:

In [116]:
clf=RandomForestClassifier(n_estimators=200,
                           min_samples_split=5,
                           min_samples_leaf=2,
                           max_features='auto',
                           max_depth=90,
                           bootstrap=False)

In [117]:
clf=model_training(clf,np.array(lsi_model[corpus_TFmatrix_train])[:,:,1],y_train)

# We test on train data
predicted_ratings = model_predictions(clf, np.array(lsi_model[corpus_TFmatrix_train])[:,:,1])
print("Classifier performances on train DB: %.2f" % reco_rate(df_dataset_train.rating, predicted_ratings), "%")

# We test on test Data
predicted_ratings_test = model_predictions(clf, np.array(lsi_model[corpus_TFmatrix_test])[:,:,1])
print("Classifier performances on test DB: %.2f"%(reco_rate(df_dataset_test.rating, predicted_ratings_test)), "%")

Classifier performances on train DB: 100.00 %
Classifier performances on test DB: 53.89 %


/home/leonardo/Documents/University/X/Courses/Cap/venv/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/home/leonardo/Documents/University/X/Courses/Cap/venv/lib/python3.7/site-packages/ipykernel_launcher.py:21: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead


### Word2Vec

In [ ]:
##To do